In [2]:
import requests as rqs
import pandas as pd
import re

In [ ]:
#Ignora questa merda

"""{'PC_Compounds': [{'id': {'id': {'cid': 2173}}, 
'atoms': {'aid': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42], 
        'element': [16, 8, 8, 8, 8, 7, 7, 7, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 
        'charge': [{'aid': 3, 'value': -1}]}, 
'bonds': {'aid1': [1, 1, 2, 3, 4, 5, 6, 6, 6, 7, 7, 7, 8, 8, 8, 9, 9, 10, 10, 10, 11, 11, 12, 12, 14, 14, 14, 15, 15, 15, 17, 18, 18, 19, 19, 20, 20, 21, 21, 22, 22, 23, 23, 24], 
        'aid2': [9, 10, 13, 16, 16, 17, 9, 11, 13, 12, 17, 34, 18, 36, 37, 12, 25, 11, 14, 15, 16, 26, 13, 27, 28, 29, 30, 31, 32, 33, 18, 19, 35, 20, 21, 22, 38, 23, 39, 24, 40, 24, 41, 42], 
        'order': [1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1]}, 
'stereo': [{'tetrahedral': {'center': 9, 'above': 1, 'top': 6, 'bottom': 12, 'below': 25, 'parity': 3, 'type': 1}}, 
           {'tetrahedral': {'center': 11, 'above': 6, 'top': 10, 'bottom': 16, 'below': 26, 'parity': 3, 'type': 1}},
            {'tetrahedral': {'center': 12, 'above': 7, 'top': 9, 'bottom': 13, 'below': 27, 'parity': 3, 'type': 1}},
            {'tetrahedral': {'center': 18, 'above': 8, 'top': 17, 'bottom': 19, 'below': 35, 'parity': 3, 'type': 1}}], 
'coords': [{'type': [1, 5, 255], 'aid': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42], 
            'conformers': [{'x': [9.2365, 6.5715, 8.8764, 10.5236, 5.3442, 8.2854, 6.5715, 5.1602, 8.2854, 9.8242, 9.2365, 7.2771, 7.2771, 10.6333, 10.6333, 9.5455, 5.605, 4.8994, 3.9329, 3.6721, 3.2273, 2.7056, 2.2608, 2, 8.1897, 8.7981, 6.7041, 10.9977, 11.1349, 10.2688, 10.2688, 11.1349, 10.9977, 6.7332, 4.4619, 4.7227, 5.7594, 4.1096, 3.389, 2.544, 1.8233, 1.4008], 
                            'y': [0.5261, -1.4957, -2.7862, -2.2509, -0.2925, -0.783, 0.9297, 2.3469, 0.217, -0.283, -1.092, 0.2212, -0.7871, 0.3048, -0.8707, -2.043, 0.6729, 1.3815, 1.1246, 0.1592, 1.8332, -0.0976, 1.5764, 0.611, 0.8296, -1.5304, -0.0155, -0.1968, 0.6693, 0.8064, -1.3723, -1.2352, -0.3691, 1.5283, 1.8208, 2.7862, 2.5061, -0.2801, 2.4317, -0.6961, 2.0157, 0.4517], 
                            'style': {'annotation': [3, 3, 3, 3, 8, 8, 8, 8, 8, 8], 'aid1': [9, 11, 12, 18, 19, 19, 20, 21, 22, 23], 'aid2': [1, 16, 7, 8, 20, 21, 22, 23, 24, 24]}}]}], 
'charge': -1, 
'props': [{'urn': {'label': 'Compound', 'name': 'Canonicalized', 'datatype': 5, 'release': '2021.05.07'}, 
                'value': {'ival': 1}}, 
          {'urn': {'label': 'Compound Complexity', 'datatype': 7, 'implementation': 'E_COMPLEXITY', 'version': '3.4.8.18', 'software': 'Cactvs', 'source': 'Xemistry GmbH', 'release': '2021.05.07'}, 
                'value': {'fval': 556}}, 
          {'urn': {'label': 'Count', 'name': 'Hydrogen Bond Acceptor', 'datatype': 5, 'implementation': 'E_NHACCEPTORS', 'version': '3.4.8.18', 'software': 'Cactvs', 'source': 'Xemistry GmbH', 'release': '2021.05.07'}, 
                'value': {'ival': 6}}, 
          {'urn': {'label': 'Count', 'name': 'Hydrogen Bond Donor', 'datatype': 5, 'implementation': 'E_NHDONORS', 'version': '3.4.8.18', 'software': 'Cactvs', 'source': 'Xemistry GmbH', 'release': '2021.05.07'}, 
                'value': {'ival': 2}}, 
          {'urn': {'label': 'Count', 'name': 'Rotatable Bond', 'datatype': 5, 'implementation': 'E_NROTBONDS', 'version': '3.4.8.18', 'software': 'Cactvs', 'source': 'Xemistry GmbH', 'release': '2021.05.07'}, 
                'value': {'ival': 3}}, 
          {'urn': {'label': 'Fingerprint', 'name': 'SubStructure Keys', 'datatype': 16, 'parameters': 'extended 2', 'implementation': 'E_SCREEN', 'version': '3.4.8.18', 'software': 'Cactvs', 'source': 'Xemistry GmbH', 'release': '2021.05.07'}, 
                'value': {'binary': '00000371E07B3800400000000000000000000000580160000000300000000580000000010000001E04100000000C28C5D804B20883C00008880221D218008200006000100888818804880A603AA09531942000649600B888071888808E00000000000000000000000000000000000000000000'}}, 
          {'urn': {'label': 'IUPAC Name', 'name': 'Allowed', 'datatype': 1, 'version': '2.7.0', 'software': 'Lexichem TK', 'source': 'OpenEye Scientific Software', 'release': '2021.05.07'}, 
                'value': {'sval': '6-[(2-amino-2-phenyl-acetyl)amino]-3,3-dimethyl-7-oxo-4-thia-1-azabicyclo[3.2.0]heptane-2-carboxylate'}}, 
          {'urn': {'label': 'IUPAC Name', 'name': 'CAS-like Style', 'datatype': 1, 'version': '2.7.0', 'software': 'Lexichem TK', 'source': 'OpenEye Scientific Software', 'release': '2021.05.07'}, 
                'value': {'sval': '6-[(2-amino-1-oxo-2-phenylethyl)amino]-3,3-dimethyl-7-oxo-4-thia-1-azabicyclo[3.2.0]heptane-2-carboxylate'}}, 
          {'urn': {'label': 'IUPAC Name', 'name': 'Markup', 'datatype': 1, 'version': '2.7.0', 'software': 'Lexichem TK', 'source': 'OpenEye Scientific Software', 'release': '2021.05.07'}, 
                'value': {'sval': '6-[(2-amino-2-phenylacetyl)amino]-3,3-dimethyl-7-oxo-4-thia-1-azabicyclo[3.2.0]heptane-2-carboxylate'}}, 
          {'urn': {'label': 'IUPAC Name', 'name': 'Preferred', 'datatype': 1, 'version': '2.7.0', 'software': 'Lexichem TK', 'source': 'OpenEye Scientific Software', 'release': '2021.05.07'}, 
                'value': {'sval': '6-[(2-amino-2-phenylacetyl)amino]-3,3-dimethyl-7-oxo-4-thia-1-azabicyclo[3.2.0]heptane-2-carboxylate'}}, 
          {'urn': {'label': 'IUPAC Name', 'name': 'Systematic', 'datatype': 1, 'version': '2.7.0', 'software': 'Lexichem TK', 'source': 'OpenEye Scientific Software', 'release': '2021.05.07'}, 
                'value': {'sval': '6-[(2-azanyl-2-phenyl-ethanoyl)amino]-3,3-dimethyl-7-oxidanylidene-4-thia-1-azabicyclo[3.2.0]heptane-2-carboxylate'}}, 
          {'urn': {'label': 'IUPAC Name', 'name': 'Traditional', 'datatype': 1, 'version': '2.7.0', 'software': 'Lexichem TK', 'source': 'OpenEye Scientific Software', 'release': '2021.05.07'}, 
                'value': {'sval': '6-[(2-amino-2-phenyl-acetyl)amino]-7-keto-3,3-dimethyl-4-thia-1-azabicyclo[3.2.0]heptane-2-carboxylate'}}, 
          {'urn': {'label': 'InChI', 'name': 'Standard', 'datatype': 1, 'version': '1.0.6', 'software': 'InChI', 'source': 'iupac.org', 'release': '2021.05.07'}, 
                'value': {'sval': 'InChI=1S/C16H19N3O4S/c1-16(2)11(15(22)23)19-13(21)10(14(19)24-16)18-12(20)9(17)8-6-4-3-5-7-8/h3-7,9-11,14H,17H2,1-2H3,(H,18,20)(H,22,23)/p-1'}}, 
          {'urn': {'label': 'InChIKey', 'name': 'Standard', 'datatype': 1, 'version': '1.0.6', 'software': 'InChI', 'source': 'iupac.org', 'release': '2021.05.07'}, 
                'value': {'sval': 'AVKUERGKIZMTKX-UHFFFAOYSA-M'}}, 
          {'urn': {'label': 'Log P', 'name': 'XLogP3', 'datatype': 7, 'version': '3.0', 'source': 'sioc-ccbg.ac.cn', 'release': '2021.05.07'}, 
                'value': {'fval': -0.5}}, 
          {'urn': {'label': 'Mass', 'name': 'Exact', 'datatype': 1, 'version': '2.1', 'software': 'PubChem', 'source': 'ncbi.nlm.nih.gov', 'release': '2021.05.07'}, 
                'value': {'sval': '348.10180224'}}, 
          {'urn': {'label': 'Molecular Formula', 'datatype': 1, 'version': '2.1', 'software': 'PubChem', 'source': 'ncbi.nlm.nih.gov', 'release': '2021.05.07'}, 
                'value': {'sval': 'C16H18N3O4S-'}}, 
          {'urn': {'label': 'Molecular Weight', 'datatype': 1, 'version': '2.1', 'software': 'PubChem', 'source': 'ncbi.nlm.nih.gov', 'release': '2021.05.07'}, 
                'value': {'sval': '348.4'}}, 
          {'urn': {'label': 'SMILES', 'name': 'Absolute', 'datatype': 1, 'version': '2.3.0', 'software': 'OEChem', 'source': 'OpenEye Scientific Software', 'release': '2024.12.12'}, 
                'value': {'sval': 'CC1(C(N2C(S1)C(C2=O)NC(=O)C(C3=CC=CC=C3)N)C(=O)[O-])C'}}, 
          {'urn': {'label': 'SMILES', 'name': 'Canonical', 'datatype': 1, 'version': '2.3.0', 'software': 'OEChem', 'source': 'OpenEye Scientific Software', 'release': '2021.05.07'}, 
                'value': {'sval': 'CC1(C(N2C(S1)C(C2=O)NC(=O)C(C3=CC=CC=C3)N)C(=O)[O-])C'}}, 
          {'urn': {'label': 'SMILES', 'name': 'Isomeric', 'datatype': 1, 'version': '2.3.0', 'software': 'OEChem', 'source': 'OpenEye Scientific Software', 'release': '2021.05.07'}, 
                'value': {'sval': 'CC1(C(N2C(S1)C(C2=O)NC(=O)C(C3=CC=CC=C3)N)C(=O)[O-])C'}}, 
          {'urn': {'label': 'Topological', 'name': 'Polar Surface Area', 'datatype': 7, 'implementation': 'E_TPSA', 'version': '3.4.8.18', 'software': 'Cactvs', 'source': 'Xemistry GmbH', 'release': '2021.05.07'}, 
                'value': {'fval': 141}}, 
          {'urn': {'label': 'Weight', 'name': 'MonoIsotopic', 'datatype': 1, 'version': '2.1', 'software': 'PubChem', 'source': 'ncbi.nlm.nih.gov', 'release': '2021.05.07'}, 
                'value': {'sval': '348.10180224'}}], 
'count': {'heavy_atom': 24, 'atom_chiral': 4, 'atom_chiral_def': 0, 'atom_chiral_undef': 4, 'bond_chiral': 0, 'bond_chiral_def': 0, 'bond_chiral_undef': 0, 'isotope_atom': 0, 'covalent_unit': 1, 'tautomers': -1}}]}"""


In [2]:
def what_we_poppin_boyz(cid):
    pattern = r"CID0+(\d+)"
    norm_cid = re.search(pattern, cid).group(1)

    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/cid/{norm_cid}/property/Title/TXT"
    response = rqs.get(url)

    if response.status_code == 200:
        return response.text.strip()
    else:
        return "i'm not popping this shit bruh"

In [8]:
print(what_we_poppin_boyz("CID00002173"))

6-{[Amino(phenyl)acetyl]amino}-3,3-dimethyl-7-oxo-4-thia-1-azabicyclo[3.2.0]heptane-2-carboxylate


In [14]:
def i_know_what_im_poppin(drug_name):
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{drug_name}/cids/TXT"
    
    response = rqs.get(url)
    
    if response.status_code == 200:
        cid = response.text.strip()
        n_zeros = 12 - len(cid) - 3
        return f"CID{''.join(['0' for n in range(n_zeros)])}{cid}"
    else:
        return f"Skibidi status - {response.status_code}: {drug_name} got no rizz fr fr"

In [34]:
print(i_know_what_im_poppin("Aspirin"))

CID000002244


In [32]:
print(what_we_poppin_boyz(i_know_what_im_poppin("Aspirin")))

Aspirin


In [3]:
drugs_n_shit = pd.read_csv("./ChChse-Decagon_polypharmacy.csv")
drugs_n_shit.head(5)

,# STITCH 1,STITCH 2,Polypharmacy Side Effect,Side Effect Name
0,CID000002173,CID000003345,C0151714,hypermagnesemia
1,CID000002173,CID000003345,C0035344,retinopathy of prematurity
2,CID000002173,CID000003345,C0004144,atelectasis
3,CID000002173,CID000003345,C0002063,alkalosis
4,CID000002173,CID000003345,C0004604,Back Ache


In [4]:
rank_drugs = pd.read_csv("./top_300_drugs.csv")
rank_drugs.head(5)

,Rank,Drug Name,Total Prescriptions (2022),Total Patients (2022),Annual Change
0,1,Atorvastatin,"109,582,746","27,935,702",0
1,2,Metformin,"86,747,907","19,536,027",0
2,3,Lisinopril,"82,513,967","20,314,304",1
3,4,Levothyroxine,"82,431,914","18,130,331",1
4,5,Amlodipine,"70,766,211","17,789,649",0


In [10]:
drugs_in_nw = drugs_n_shit["# STITCH 1"].unique().tolist()  
drugs_in_nw += drugs_n_shit["STITCH 2"].unique().tolist()
drugs_in_nw = list(set(drugs_in_nw))

print(drugs_in_nw)
print(len(drugs_in_nw))


['CID000004894', 'CID000060815', 'CID000003780', 'CID000004205', 'CID000002477', 'CID000002673', 'CID000005297', 'CID000443871', 'CID000003148', 'CID000003016', 'CID000004499', 'CID000004158', 'CID000060164', 'CID000000598', 'CID000005039', 'CID000154059', 'CID000003821', 'CID000005584', 'CID000005372', 'CID000002973', 'CID000002232', 'CID000005734', 'CID000003488', 'CID000002732', 'CID000148211', 'CID000004539', 'CID000003403', 'CID000006691', 'CID000005505', 'CID000005035', 'CID000003440', 'CID000005426', 'CID000002141', 'CID000003750', 'CID000002708', 'CID005282226', 'CID000213039', 'CID000003675', 'CID000003249', 'CID000002249', 'CID000004513', 'CID000004595', 'CID000001690', 'CID000027686', 'CID000002676', 'CID000003261', 'CID000002349', 'CID000004543', 'CID000005376', 'CID000005002', 'CID000005029', 'CID000003642', 'CID000005454', 'CID000104741', 'CID000004449', 'CID000003957', 'CID000005195', 'CID000000853', 'CID000003958', 'CID000002662', 'CID000005726', 'CID000004946', 'CID000

In [12]:
drugs_present_in_both = []

In [15]:
for idx, row in rank_drugs.iterrows():
    drug = row["Drug Name"]
    rank = row["Rank"]
    if i_know_what_im_poppin(drug) in drugs_in_nw:
        drugs_present_in_both.append([rank, drug])

ranked_drugs_in_nw = pd.DataFrame(drugs_present_in_both,columns=["Rank", "Drug Name"])

In [16]:
print(ranked_drugs_in_nw)

     Rank            Drug Name
0       2            Metformin
1       5           Amlodipine
2       6           Metoprolol
3       7            Albuterol
4       8             Losartan
..    ...                  ...
100   270         Fluorouracil
101   276  Medroxyprogesterone
102   291     Chlorpheniramine
103   295          Guaifenesin
104   298              Azelate

[105 rows x 2 columns]


In [17]:
ranked_drugs_in_nw.to_csv("./ranked_drugs_matching.csv")